In [8]:
import sympy as sp

def commutator(A, B):
    return A * B - B * A

def nested_commutator(A, B, n):
    if n == 1:
        return commutator(A, B)
    else:
        return commutator(A, nested_commutator(A, B, n-1))
def each_level_its_own_subspace(dim):
    """
    Obtain the indices of the subspace to consider.

    Args:
        dim (int): The dimension of the subspace.

    Returns:
        list: A list of lists, each containing a single index of the subspace to consider.
    """
    # Define the indices of the subspace to consider
    subspace_indices = [[i] for i in range(dim)]
    return subspace_indices

def create_subspace_projectors(dim, subspace_indices):
    """
    Create projectors for the given subspaces using SymPy.

    Args:
        dim (int): The total dimension of the Hilbert space.
        subspace_indices (list): A list of lists, each containing the indices of a subspace.

    Returns:
        list: A list of projectors (SymPy matrices) for the given subspaces.
    """
    projectors = []
    
    for subspace in subspace_indices:
        # Initialize the projector matrix P as a zero matrix of size dim x dim
        P = sp.zeros(dim, dim)
        
        # Create the basis states and compute the projector for each index in the subspace
        for idx in subspace:
            # Create a basis state (column vector) with 1 at the index 'idx'
            state = sp.zeros(dim, 1)
            state[idx, 0] = 1
            
            # Compute the rank-1 projector for this basis state: |state><state|
            P += state * state.T  # Outer product in SymPy is `*`
        
        # Add the projector to the list
        projectors.append(P)
    
    return projectors

def swt_subspace(H0, V, subspace_indices):
    """
    Compute the Schrieffer-Wolff transformation on a subspace of the Hamiltonian.

    Args:
        H0 (sp.Matrix): The unperturbed Hamiltonian.
        V (sp.Matrix): The perturbation Hamiltonian.
        subspace_indices (list): The indices of the subspace to consider.

    Returns:
        tuple: The Schrieffer-Wolff transformation operators (S1, S2, S3) and 
               the transformed Hamiltonian components (H1, H2, H3, H4).
    """
    dim = H0.shape[0]
    projectors = create_subspace_projectors(dim, subspace_indices)
    Vd = sp.zeros(dim, dim)
    for P in projectors:
        Vd += P * V * P
    Vod = V - Vd
    # Compute the energy differences in the subspace
    delta = sp.Matrix([[H0[i,i] - H0[j,j] if i != j else 1 for j in range(dim)] for i in range(dim)])

    # Compute the Schrieffer-Wolff transformation components
    H1 = Vd
    S1 = sp.Matrix([[Vod[i,j] / delta[i,j] for j in range(dim)] for i in range(dim)])
    H2 = sp.Rational(1, 2) * commutator(S1, Vod)
    
    S2 = sp.Matrix([[-commutator(Vd, S1)[i,j] / delta[i,j] for j in range(dim)] for i in range(dim)])
    # H3 = sp.Rational(1, 2) * commutator(S2, Vod)
    # S3 = sp.Matrix([[(commutator(S2, Vd)[i,j] + sp.Rational(1, 3) * nested_commutator(S1, Vod, 2)[i,j]) / delta[i,j] for j in range(dim)] for i in range(dim)])
    # H4 = sp.Rational(1, 2) * commutator(S3, Vod) - sp.Rational(1, 24) * nested_commutator(S1, Vod, 3)

    return [S1,S2], [H1, H2]

In [9]:
from sympy.physics.quantum import TensorProduct
from sympy import symbols, sqrt
nc=5
ac = sp.zeros(nc)
acd = sp.zeros(nc)

# Fill the matrices using sympy functions
for i in range(nc-1):
    ac[i, i+1] = sqrt(i+1)
    acd[i+1, i] = sqrt(i+1)


In [19]:
deltaq, deltac,  = symbols('delta_q delta_a', real=True)
alphaq  = symbols('alpha_q', real=True)
g = symbols('g', real=True)
A = symbols('A', real=True)
nq = 5
nc = 5

In [20]:
V = g*(TensorProduct(ac,acd)+TensorProduct(acd,ac))

ENN = [
    l*deltaq + i*deltac + l*(l-1) * alphaq / 2 
    for l in range(nc)
    for i in range(nq)
]


In [21]:
H0 = sp.diag(*ENN)
indices = each_level_its_own_subspace(nq*nc)
result = swt_subspace(H0, V, indices)

In [22]:
H1 = result[1][0]
H2 = result[1][1]

S1 = result[0][0]

In [23]:
(commutator(S1,TensorProduct(acd,sp.eye(nc)))*commutator(S1,TensorProduct(ac,sp.eye(nc)))).applyfunc(sp.simplify)

Matrix([
[0,                           0,                                                                     0,                                                                             0,                                                                             0, 0,                                                                                               0,                                                                                                                                           0,                                                                                                                                   0,                                                                                                                                           0,                                                                           0,                                                                                                                                          

In [24]:
commutator(S1,TensorProduct(ac,sp.eye(nc))).applyfunc(sp.simplify)

Matrix([
[0, g/(delta_a - delta_q),                             0,                             0,                       0, 0,                                                                                    0,                                                                                            0,                                                                                            0,                                                                                      0,                                                                     0,                                                                                               0,                                                                                                       0,                                                                                                       0,                                                                                                 0,                                    

In [45]:
def expand_and_simplify_matrix(matrix, var, point, order):
    expanded_matrix = matrix.applyfunc(sp.simplify)
    expanded_matrix = expanded_matrix.applyfunc(lambda term: term.series(var, point, order).removeO().simplify())
    return expanded_matrix
matrix = commutator(S1,TensorProduct(ac,sp.eye(nc)))
expanded_matrix = expand_and_simplify_matrix(matrix, alphaq, 0, 2)

In [46]:
expanded_matrix

Matrix([
[0, g/(delta_a - delta_q),                             0,                             0,                       0, 0,                                                        0,                                                                0,                                                                0,                                                          0,                                         0,                                                        0,                                                                0,                                                                0,                                                          0,                                         0,                                                        0,                                                                0,                                                                0,                                                          0,                                       